### 3. **Практика: Реализация фильтрации и сортировки для бизнес-задачи**


#### **Кейс: разработка фильтров и сортировки для панели управления**

Предположим, что необходимо создать панель управления для администратора интернет-магазина, которая отображает:
1. Список пользователей с количеством заказов за последние 30 дней.
2. Фильтрацию продуктов по категориям и цене.
3. Сортировку продуктов по популярности (например, количеству заказов).

##### **1. Настройка моделей**

Предположим, у нас уже созданы модели `User`, `Product`, `Order`, и `OrderItem` для реализации базы данных интернет-магазина. В следующих примерах будем работать с этими моделями:

**Файл: `models.py` в папке приложения (например, `shop/models.py`)**

```python
from django.db import models
from django.utils import timezone
from django.contrib.auth.models import User

class Product(models.Model):
    name = models.CharField(max_length=100)
    category = models.CharField(max_length=50)
    price = models.DecimalField(max_digits=10, decimal_places=2)

class Order(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    created_at = models.DateTimeField(auto_now_add=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    quantity = models.IntegerField(default=1)
```

##### **2. Вывод списка пользователей с количеством заказов за последние 30 дней**

Для создания фильтра, который отображает пользователей с количеством заказов за последние 30 дней, используем аннотации и агрегации.

**Файл: `views.py` в папке приложения (например, `shop/views.py`)**

```python
from django.utils import timezone
from django.db.models import Count, Q
from .models import User, Order

def recent_orders_view(request):
    thirty_days_ago = timezone.now() - timezone.timedelta(days=30)
    
    # Аннотируем количество заказов за последние 30 дней для каждого пользователя
    users_with_recent_orders = User.objects.annotate(
        recent_order_count=Count('order', filter=Q(order__created_at__gte=thirty_days_ago))
    ).filter(recent_order_count__gt=0)

    return render(request, 'shop/recent_orders.html', {'users': users_with_recent_orders})
```

Здесь:
   - `annotate` используется для добавления дополнительного поля `recent_order_count`, в котором хранится количество заказов за последние 30 дней.
   - Мы фильтруем `User`-записи, у которых `recent_order_count` больше нуля.

**Файл: `templates/shop/recent_orders.html`**

```html
<h2>Пользователи с заказами за последние 30 дней</h2>
<ul>
    {% for user in users %}
        <li>{{ user.username }} - Заказов: {{ user.recent_order_count }}</li>
    {% endfor %}
</ul>
```

##### **3. Фильтрация продуктов по категориям и цене**

Добавим фильтры по категориям и цене, что позволит администрации просматривать продукты на основе указанных параметров. Здесь мы используем `GET`-параметры, чтобы отобразить выбранные пользователем фильтры.

**Файл: `views.py`**

```python
from django.db.models import Q
from .models import Product

def product_filter_view(request):
    category = request.GET.get('category')
    min_price = request.GET.get('min_price')
    max_price = request.GET.get('max_price')
    
    products = Product.objects.all()
    
    if category:
        products = products.filter(category=category)
    if min_price:
        products = products.filter(price__gte=min_price)
    if max_price:
        products = products.filter(price__lte=max_price)
    
    return render(request, 'shop/product_list.html', {'products': products})
```

**Файл: `templates/shop/product_list.html`**

```html
<h2>Фильтр товаров</h2>
<form method="GET">
    <select name="category">
        <option value="">Все категории</option>
        <option value="electronics">Электроника</option>
        <option value="clothing">Одежда</option>
        <!-- Добавьте другие категории -->
    </select>
    <input type="number" name="min_price" placeholder="Мин. цена">
    <input type="number" name="max_price" placeholder="Макс. цена">
    <button type="submit">Применить фильтр</button>
</form>

<h2>Список товаров</h2>
<ul>
    {% for product in products %}
        <li>{{ product.name }} - Категория: {{ product.category }} - Цена: {{ product.price }}</li>
    {% endfor %}
</ul>
```

##### **4. Сортировка продуктов по популярности**

Чтобы отсортировать продукты по популярности (например, количеству заказов), добавим аннотацию, вычисляющую количество раз, когда продукт был заказан, и сортировку на основе этого значения.

**Файл: `views.py`**

```python
from django.db.models import Count
from .models import Product, OrderItem

def popular_products_view(request):
    products = Product.objects.annotate(
        order_count=Count('orderitem')
    ).order_by('-order_count')  # сортировка по количеству заказов в убывающем порядке

    return render(request, 'shop/popular_products.html', {'products': products})
```

**Файл: `templates/shop/popular_products.html`**

```html
<h2>Популярные товары</h2>
<ul>
    {% for product in products %}
        <li>{{ product.name }} - Заказан: {{ product.order_count }} раз</li>
    {% endfor %}
</ul>
```

##### **5. Объединение фильтров и сортировки с интерфейсом**

Добавим форму с выбором фильтров и сортировки, которые администратор может настраивать на одном экране.

**Файл: `views.py`**

```python
def combined_filter_view(request):
    category = request.GET.get('category')
    min_price = request.GET.get('min_price')
    max_price = request.GET.get('max_price')
    sort = request.GET.get('sort', '-order_count')
    
    products = Product.objects.annotate(
        order_count=Count('orderitem')
    )
    
    if category:
        products = products.filter(category=category)
    if min_price:
        products = products.filter(price__gte=min_price)
    if max_price:
        products = products.filter(price__lte=max_price)
    
    products = products.order_by(sort)
    
    return render(request, 'shop/combined_filter.html', {'products': products})
```

**Файл: `templates/shop/combined_filter.html`**

```html
<h2>Фильтр и сортировка товаров</h2>
<form method="GET">
    <select name="category">
        <option value="">Все категории</option>
        <option value="electronics">Электроника</option>
        <option value="clothing">Одежда</option>
        <!-- Добавьте другие категории -->
    </select>
    <input type="number" name="min_price" placeholder="Мин. цена">
    <input type="number" name="max_price" placeholder="Макс. цена">
    
    <select name="sort">
        <option value="-order_count">По популярности</option>
        <option value="price">Цена (по возрастанию)</option>
        <option value="-price">Цена (по убыванию)</option>
    </select>
    
    <button type="submit">Применить</button>
</form>

<h2>Список товаров</h2>
<ul>
    {% for product in products %}
        <li>{{ product.name }} - Заказан: {{ product.order_count }} раз - Цена: {{ product.price }}</li>
    {% endfor %}
</ul>
```